# Carga de librerias

In [3]:
from elasticsearch import Elasticsearch
from json import dumps
import os

es = Elasticsearch("http://127.0.0.1:9200")

print(f"Connected to ElasticSearch cluster `{es.info().body['cluster_name']}`")

Connected to ElasticSearch cluster `docker-cluster`


# Inserción de 1 millón de documentos

En esta sección se revisará la carga y el tamaño de almacenamiento de 1 millón de documentos indexados en la base de datos.

## Carga de documentos

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-elastic.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-elastic.py -n 1

```

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/elastic-insert/load_1M_documents_elastic.png)

Como se puede ver en la imagen superior, cargar 1 millón de documentos ha tomado un tiempo de **4 minutos y 26.16 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en el índice ```extracto_cuenta_1m``` de la siguiente manera.

In [33]:
cnt = es.cat.count(index="extracto_cuenta_1m", format= "json")[0]['count']
print(f"Actualmente existen {cnt} documentos en el índice 'extracto_cuenta_1m'")

Actualmente existen 1000000 documentos en el índice 'extracto_cuenta_1m'


## Tamaño de almacenamiento de datos

Para encontrar el tamaño de los archivos JSON en su forma bruta, antes de cargarse a la base de datos, se utilizará el siguiente script:

In [ ]:
file_sizes = []
n = 1

for i, json_file in enumerate(sorted(os.listdir("../json-generated-data-elastic"))):
    file_stat = os.stat(os.path.join("../json-generated-data-elastic", json_file))
    file_sizes.append(file_stat)

    if i+1 == n:
        break

for file in file_sizes:
    print(f'Documento {json_file} con tamaño en memoria de {file.st_size} bytes')

Documento random-generated-data-1.json con tamaño en memoria de 188495887 bytes


Para obtener las estadísticas de uso de disco por parte del índice ```extracto_cuenta_1m``` en la base de datos ejecutaremos el siguiente comando:

In [10]:
r = dumps(es.indices.disk_usage(index="extracto_cuenta_1m", run_expensive_tasks=True)['extracto_cuenta_1m'], indent=1)
print(r[:300])
print("...")

{
 "store_size": "138.3mb",
 "store_size_in_bytes": 145089787,
 "all_fields": {
  "total": "137.5mb",
  "total_in_bytes": 144187007,
  "inverted_index": {
   "total": "19mb",
   "total_in_bytes": 19932341
  },
  "stored_fields": "87.7mb",
  "stored_fields_in_bytes": 91994865,
  "doc_values": "12.8mb",
  "doc_values_in_bytes": 13503952,
  "points": "15.9mb",
  "points_in_bytes": 16755849,
  "norms": "1.9mb",
  "norms_in_bytes": 2000000,
  "term_vectors": "0b",
  "term_vectors_in_bytes": 0,
  "knn_vectors": "0b",
  "knn_vectors_in_bytes": 0
 },
 "fields": {
  "_id": {
   "total": "15.5mb",
   "total_in_bytes": 16300151,
   "inverted_index": {
    "total": "5.1mb",
    "total_in_bytes": 5348000
   },
   "stored_fields": "10.4mb",
   "stored_fields_in_bytes": 10952151,
   "doc_values": "0b",
   "doc_values_in_bytes": 0,
   "points": "0b",
   "points_in_bytes": 0,
   "norms": "0b",
   "norms_in_bytes": 0,
   "term_vectors": "0b",
   "term_vectors_in_bytes": 0,
   "knn_vectors": "0b",
   "kn

In [12]:
r = dumps(es.indices.stats(metric='store', human=True, index="extracto_cuenta_1m"), indent = 1)
print(r)

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_all': {'primaries': {'store': {'size': '138.3mb', 'size_in_bytes': 145089787, 'total_data_set_size': '138.3mb', 'total_data_set_size_in_bytes': 145089787, 'reserved': '0b', 'reserved_in_bytes': 0}}, 'total': {'store': {'size': '138.3mb', 'size_in_bytes': 145089787, 'total_data_set_size': '138.3mb', 'total_data_set_size_in_bytes': 145089787, 'reserved': '0b', 'reserved_in_bytes': 0}}}, 'indices': {'extracto_cuenta_1m': {'uuid': 'eyXzz3y6Tyu1NSdvSW60xQ', 'health': 'yellow', 'status': 'open', 'primaries': {'store': {'size': '138.3mb', 'size_in_bytes': 145089787, 'total_data_set_size': '138.3mb', 'total_data_set_size_in_bytes': 145089787, 'reserved': '0b', 'reserved_in_bytes': 0}}, 'total': {'store': {'size': '138.3mb', 'size_in_bytes': 145089787, 'total_data_set_size': '138.3mb', 'total_data_set_size_in_bytes': 145089787, 'reserved': '0b', 'reserved_in_bytes': 0}}}}})

Extrayendo información importante de los datos recién obtenidos. El tamaño en almacenamiento aproximado de 1 documento de 1 millón de registros con un peso en bruto de 179.76 mb (en escala base binario $2^{20}$), una vez comprimido en la base de datos es **137.50 mb** (en escala base binario $2^{20}$). Esto nos da un radio de compresión de 0.7649 y una reducción de tamaño en memoria del 23.51%.

![Cargando 1 millón de documentos](../imagenes-de-soporte/elastic-insert/storage_kibana_1M_documents.png)

------------------------

# Inserción de 10 millones de documentos

En esta sección se revisará la carga y el tamaño de almacenamiento de 10 millones de documentos indexados en la base de datos.

## Carga de documentos

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-elastic.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-elastic.py -n 10

```

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/elastic-insert/load_10M_documents_elastic.png)

Como se puede ver en la imagen superior, cargar 10 millones de documentos ha tomado un tiempo de **48 minutos y 52.61 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en el índice ```extracto_cuenta_10m``` de la siguiente manera.

In [36]:
cnt = es.cat.count(index="extracto_cuenta_10m", format= "json")[0]['count']
print(f"Actualmente existen {cnt} documentos en el índice 'extracto_cuenta_10m'")

Actualmente existen 10000000 documentos en el índice 'extracto_cuenta_10m'


## Tamaño de almacenamiento de datos

Para obtener las estadísticas de uso de disco por parte del índice ```extracto_cuenta_10m``` en la base de datos ejecutaremos el siguiente comando:

In [40]:
r = dumps(es.indices.disk_usage(index="extracto_cuenta_10m", run_expensive_tasks=True)['extracto_cuenta_10m'], indent=1)
print(r[:500])
print("...")

{
 "store_size": "1.3gb",
 "store_size_in_bytes": 1440202391,
 "all_fields": {
  "total": "1.3gb",
  "total_in_bytes": 1438808499,
  "inverted_index": {
   "total": "191.3mb",
   "total_in_bytes": 200694994
  },
  "stored_fields": "876.7mb",
  "stored_fields_in_bytes": 919322081,
  "doc_values": "128.7mb",
  "doc_values_in_bytes": 135007131,
  "points": "156.1mb",
  "points_in_bytes": 163784293,
  "norms": "19mb",
  "norms_in_bytes": 20000000,
  "term_vectors": "0b",
  "term_vectors_in_bytes": 0
...


Extrayendo información importante de los datos recién obtenidos. El tamaño de almacenamiento de 10 documento de 1 millón de registros con un peso en bruto de 1.9 gb, **una vez comprimido en la base de datos es 1.34 gb**. Esto nos da un radio de compresión de 1.42 y una reducción de tamaño en memoria del 29.47%.

![Cargando 1 millón de documentos](../imagenes-de-soporte/elastic-insert/kibana_10M_documents.png)

In [15]:

mapping = {
        "properties": {
            "fecha": {
                "type": "date",
                "format": "yyyy-MM-dd HH:mm:ss",
                "index": "false"
            },
            "agencia": {
                "type": "keyword",
                "index": "false"
            },
            "monto": {
                "type": "float",
                "index": "false"
            },
            "descripcion": {
                "type": "text",
                "index": "false"
            },
            "saldo": {
                "type": "float",
                "index": "false"
            },
            "nota": {
                "type": "text",
                "index": "false"
            }
        }
}

r = es.indices.create(
    index="extracto_cuenta_test",
    mappings = mapping
)


In [5]:
import time
from elasticsearch import helpers
n=1
start_time = time.time()
for i, json_file in enumerate(sorted(os.listdir("../json-generated-data-elastic"))):
    with open(os.path.join("../json-generated-data-elastic", json_file)) as file:
        file_dict = file.readlines()
    
    print(f'Cargando el documento {i+1}/{n}')
    
    
    helpers.bulk(es, file_dict, index=f"extracto_cuenta_test")
    
    
    if i+1 == n:
        break

end_time = time.time()

Cargando el documento 1/1


In [4]:
es.indices.stats(metric='store', human=True, index="extracto_cuenta_test")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_all': {'primaries': {'store': {'size': '225b', 'size_in_bytes': 225, 'total_data_set_size': '225b', 'total_data_set_size_in_bytes': 225, 'reserved': '0b', 'reserved_in_bytes': 0}}, 'total': {'store': {'size': '225b', 'size_in_bytes': 225, 'total_data_set_size': '225b', 'total_data_set_size_in_bytes': 225, 'reserved': '0b', 'reserved_in_bytes': 0}}}, 'indices': {'extracto_cuenta_test': {'uuid': '2REKFn6qTLy0afgvHMaqJw', 'health': 'yellow', 'status': 'open', 'primaries': {'store': {'size': '225b', 'size_in_bytes': 225, 'total_data_set_size': '225b', 'total_data_set_size_in_bytes': 225, 'reserved': '0b', 'reserved_in_bytes': 0}}, 'total': {'store': {'size': '225b', 'size_in_bytes': 225, 'total_data_set_size': '225b', 'total_data_set_size_in_bytes': 225, 'reserved': '0b', 'reserved_in_bytes': 0}}}}})

In [6]:
es.indices.stats(metric='store', human=True, index="extracto_cuenta_test")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_all': {'primaries': {'store': {'size': '105.6mb', 'size_in_bytes': 110769373, 'total_data_set_size': '105.6mb', 'total_data_set_size_in_bytes': 110769373, 'reserved': '0b', 'reserved_in_bytes': 0}}, 'total': {'store': {'size': '105.6mb', 'size_in_bytes': 110769373, 'total_data_set_size': '105.6mb', 'total_data_set_size_in_bytes': 110769373, 'reserved': '0b', 'reserved_in_bytes': 0}}}, 'indices': {'extracto_cuenta_test': {'uuid': '2REKFn6qTLy0afgvHMaqJw', 'health': 'yellow', 'status': 'open', 'primaries': {'store': {'size': '105.6mb', 'size_in_bytes': 110769373, 'total_data_set_size': '105.6mb', 'total_data_set_size_in_bytes': 110769373, 'reserved': '0b', 'reserved_in_bytes': 0}}, 'total': {'store': {'size': '105.6mb', 'size_in_bytes': 110769373, 'total_data_set_size': '105.6mb', 'total_data_set_size_in_bytes': 110769373, 'reserved': '0b', 'reserved_in_bytes': 0}}}}})

In [9]:
r = dumps(es.indices.disk_usage(index="extracto_cuenta_test", run_expensive_tasks=True)['extracto_cuenta_test'], indent=1)
print(r)

{
 "store_size": "107.7mb",
 "store_size_in_bytes": 112948520,
 "all_fields": {
  "total": "107mb",
  "total_in_bytes": 112223888,
  "inverted_index": {
   "total": "5.2mb",
   "total_in_bytes": 5527165
  },
  "stored_fields": "87.7mb",
  "stored_fields_in_bytes": 92016342,
  "doc_values": "12.9mb",
  "doc_values_in_bytes": 13567494,
  "points": "1mb",
  "points_in_bytes": 1112887,
  "norms": "0b",
  "norms_in_bytes": 0,
  "term_vectors": "0b",
  "term_vectors_in_bytes": 0,
  "knn_vectors": "0b",
  "knn_vectors_in_bytes": 0
 },
 "fields": {
  "_id": {
   "total": "15.7mb",
   "total_in_bytes": 16521036,
   "inverted_index": {
    "total": "5.2mb",
    "total_in_bytes": 5527165
   },
   "stored_fields": "10.4mb",
   "stored_fields_in_bytes": 10993871,
   "doc_values": "0b",
   "doc_values_in_bytes": 0,
   "points": "0b",
   "points_in_bytes": 0,
   "norms": "0b",
   "norms_in_bytes": 0,
   "term_vectors": "0b",
   "term_vectors_in_bytes": 0,
   "knn_vectors": "0b",
   "knn_vectors_in_by

In [12]:
es.indices.stats(metric='store', human=True, index="extracto_cuenta_30m")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_all': {'primaries': {'store': {'size': '3.1gb', 'size_in_bytes': 3364925723, 'total_data_set_size': '3.1gb', 'total_data_set_size_in_bytes': 3364925723, 'reserved': '0b', 'reserved_in_bytes': 0}}, 'total': {'store': {'size': '3.1gb', 'size_in_bytes': 3364925723, 'total_data_set_size': '3.1gb', 'total_data_set_size_in_bytes': 3364925723, 'reserved': '0b', 'reserved_in_bytes': 0}}}, 'indices': {'extracto_cuenta_30m': {'uuid': '9VBDsOpnTrizdpSu-e_aBA', 'health': 'yellow', 'status': 'open', 'primaries': {'store': {'size': '3.1gb', 'size_in_bytes': 3364925723, 'total_data_set_size': '3.1gb', 'total_data_set_size_in_bytes': 3364925723, 'reserved': '0b', 'reserved_in_bytes': 0}}, 'total': {'store': {'size': '3.1gb', 'size_in_bytes': 3364925723, 'total_data_set_size': '3.1gb', 'total_data_set_size_in_bytes': 3364925723, 'reserved': '0b', 'reserved_in_bytes': 0}}}}})

In [19]:
r = dumps(es.indices.disk_usage(index="extracto_cuenta_30m", run_expensive_tasks=True)['extracto_cuenta_1m'], indent=1)
print(r)

ConnectionTimeout: Connection timed out

In [24]:

mapping = {
        "properties": {
            "fecha": {
                "type": "date",
                "format": "yyyy-MM-dd HH:mm:ss",
                "index": "true"
            },
            "agencia": {
                "type": "keyword",
                "index": "true"
            },
            "monto": {
                "type": "float",
                "index": "true"
            },
            "descripcion": {
                "type": "text",
                "index": "true"
            },
            "saldo": {
                "type": "float",
                "index": "true"
            },
            "nota": {
                "type": "text",
                "index": "true"
            }
        }
}

r = es.indices.create(
    index="extracto_cuenta_30m_reindexed",
    mappings = mapping
)

In [32]:
source = {
    "index": "extracto_cuenta_30m",
    "query": {}
}
dest =  {"index": "extracto_cuenta_30m_reindexed"}

In [35]:
r = es.options(request_timeout=30).reindex(source=source, dest=dest)

ConnectionTimeout: Connection timed out